<a href="https://colab.research.google.com/github/iamprabhanjan/tmp2/blob/main/MiniGPT4_video_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone -b dev https://github.com/camenduru/MiniGPT4-video
%cd /content/MiniGPT4-video

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Vision-CAIR/MiniGPT4-Video/resolve/main/checkpoints/video_llama_checkpoint_last.pth -d /content/MiniGPT4-video/pretrained_models -o video_llama_checkpoint_last.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Vision-CAIR/MiniGPT4-Video/resolve/main/checkpoints/video_mistral_checkpoint_last.pth -d /content/MiniGPT4-video/pretrained_models -o video_mistral_checkpoint_last.pth

!pip install -q webvtt-py==0.4.6 omegaconf==2.3.0 iopath==0.1.10 timm==0.6.13  webdataset==0.2.48 visual-genome==1.1.1 decord==0.6.0 pysrt==1.1.2 peft==0.2.0 wandb==0.14.2 openai==0.28.0 pycocoevalcap==1.2
!pip install -q pytubefix transformers==4.37.2 moviepy==1.0.3 nltk==3.8.1 soundfile==0.12.1 sentencepiece==0.1.97 bitsandbytes==0.42.0 accelerate==0.25.0 opencv-python==4.7.0.72 scikit-image==0.22.0

/content
Cloning into 'MiniGPT4-video'...
remote: Enumerating objects: 756, done.
remote: Counting objects: 100% (215/215), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 756 (delta 180), reused 149 (delta 149), pack-reused 541 (from 1)
Receiving objects: 100% (756/756), 35.83 MiB | 20.59 MiB/s, done.
Resolving deltas: 100% (348/348), done.
/content/MiniGPT4-video
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselect

In [5]:
import os
import torch
import webvtt
import cv2
from minigpt4.common.eval_utils import init_model
from minigpt4.conversation.conversation import CONV_VISION
from torchvision import transforms
from tqdm import tqdm
import soundfile as sf
import moviepy.editor as mp
from PIL import Image
from moviepy.editor import VideoFileClip
import random
import numpy as np
import torch.backends.cudnn as cudnn

class Options:
    def __init__(self):
        self.cfg_path = "/content/MiniGPT4-video/test_configs/llama2_test_config.yaml"
        self.ckpt = '/content/MiniGPT4-video/pretrained_models/video_llama_checkpoint_last.pth'
        #self.cfg_path = "/content/MiniGPT4-video/test_configs/mistral_test_config.yaml"
        #self.ckpt = '/content/MiniGPT4-video/pretrained_models/video_mistral_checkpoint_last.pth'
        self.add_subtitles = False
        self.question = None
        self.video_path = None
        self.max_new_tokens = 512
        self.lora_r = 64
        self.lora_alpha = 16
        self.options = None

args=Options()

def prepare_input(vis_processor,video_path,subtitle_path,instruction):
    cap = cv2.VideoCapture(video_path)
    if subtitle_path is not None:
        # Load the VTT subtitle file
        vtt_file = webvtt.read(subtitle_path)
        print("subtitle loaded successfully")
        clip = VideoFileClip(video_path)
        total_num_frames = int(clip.duration * clip.fps)
        # print("Video duration = ",clip.duration)
        clip.close()
    else :
        # calculate the total number of frames in the video using opencv
        total_num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    max_images_length = 45
    max_sub_len = 400
    images = []
    frame_count = 0
    sampling_interval = int(total_num_frames / max_images_length)
    if sampling_interval == 0:
        sampling_interval = 1
    img_placeholder = ""
    subtitle_text_in_interval = ""
    history_subtitles = {}
    raw_frames=[]
    number_of_words=0
    transform=transforms.Compose([transforms.ToPILImage(),])
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Find the corresponding subtitle for the frame and combine the interval subtitles into one subtitle
        # we choose 1 frame for every 2 seconds,so we need to combine the subtitles in the interval of 2 seconds
        if subtitle_path is not None:
            for subtitle in vtt_file:
                sub=subtitle.text.replace('\n',' ')
                if (subtitle.start_in_seconds <= (frame_count / int(clip.fps)) <= subtitle.end_in_seconds) and sub not in subtitle_text_in_interval:
                    if not history_subtitles.get(sub,False):
                        subtitle_text_in_interval+=sub+" "
                    history_subtitles[sub]=True
                    break
        if frame_count % sampling_interval == 0:
            raw_frames.append(Image.fromarray(cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2RGB)))
            frame = transform(frame[:,:,::-1]) # convert to RGB
            frame = vis_processor(frame)
            images.append(frame)
            img_placeholder += '<Img><ImageHere>'
            if subtitle_path is not None and subtitle_text_in_interval != "" and number_of_words< max_sub_len:
                img_placeholder+=f'<Cap>{subtitle_text_in_interval}'
                number_of_words+=len(subtitle_text_in_interval.split(' '))
                subtitle_text_in_interval = ""
        frame_count += 1

        if len(images) >= max_images_length:
            break
    cap.release()
    cv2.destroyAllWindows()
    if len(images) == 0:
        # skip the video if no frame is extracted
        return None,None
    images = torch.stack(images)
    instruction = img_placeholder + '\n' + instruction
    return images,instruction
def extract_audio(video_path, audio_path):
    video_clip = mp.VideoFileClip(video_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(audio_path, codec="libmp3lame", bitrate="320k")

def generate_subtitles(video_path):
    video_id=video_path.split('/')[-1].split('.')[0]
    audio_path = f"workspace/inference_subtitles/mp3/{video_id}"+'.mp3'
    os.makedirs("workspace/inference_subtitles/mp3",exist_ok=True)
    if existed_subtitles.get(video_id,False):
        return f"workspace/inference_subtitles/{video_id}"+'.vtt'
    try:
        extract_audio(video_path,audio_path)
        print("successfully extracted")
        os.system(f"whisper {audio_path}  --language English --model large --output_format vtt --output_dir workspace/inference_subtitles")
        # remove the audio file
        os.system(f"rm {audio_path}")
        print("subtitle successfully generated")
        return f"workspace/inference_subtitles/{video_id}"+'.vtt'
    except Exception as e:
        print("error",e)
        print("error",video_path)
        return None

def run (video_path,instruction,model,vis_processor,gen_subtitles=True):
    if gen_subtitles:
        subtitle_path=generate_subtitles(video_path)
    else :
        subtitle_path=None
    prepared_images,prepared_instruction=prepare_input(vis_processor,video_path,subtitle_path,instruction)
    if prepared_images is None:
        return "Video cann't be open ,check the video path again"
    length=len(prepared_images)
    prepared_images=prepared_images.unsqueeze(0)
    conv = CONV_VISION.copy()
    conv.system = ""
    # if you want to make conversation comment the 2 lines above and make the conv is global variable
    conv.append_message(conv.roles[0], prepared_instruction)
    conv.append_message(conv.roles[1], None)
    prompt = [conv.get_prompt()]
    answers = model.generate(prepared_images, prompt, max_new_tokens=args.max_new_tokens, do_sample=True, lengths=[length],num_beams=1)
    return answers[0]

def setup_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    cudnn.benchmark = False
    cudnn.deterministic = True

import yaml
with open('test_configs/llama2_test_config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
seed=config['run']['seed']
print("seed",seed)

model, vis_processor = init_model(args)
conv = CONV_VISION.copy()
conv.system = ""
inference_subtitles_folder="inference_subtitles"
os.makedirs(inference_subtitles_folder,exist_ok=True)
existed_subtitles={}
for sub in os.listdir(inference_subtitles_folder):
    existed_subtitles[sub.split('.')[0]]=True

seed 42
Initialization Model
--------------
model arch mini_gpt4_llama_v2
model cls <class 'minigpt4.models.mini_gpt4_llama_v2.MiniGPT4_llama_v2'>
dataset name video_chatgpt
Llama model
token pooling True
vit precision fp16
freeze the vision encoder
Loading VIT Done
Loading LLAMA


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

  warnings.warn(

  warnings.warn(



trainable params: 33554432 || all params: 6771970048 || trainable%: 0.49548996469513035
Loading LLAMA Done


  ckpt = torch.load(ckpt_path, map_location="cpu")



Load Minigpt-4-LLM Checkpoint: /content/MiniGPT4-video/pretrained_models/video_llama_checkpoint_last.pth
{'name': 'blip2_image_train', 'image_size': 224}
Initialization Finished


In [10]:
args.video_path = '/content/videott.mp4'
args.question = "What's this video talking about?"
args.add_subtitles = False

pred=run(args.video_path, args.question, model, vis_processor, gen_subtitles=args.add_subtitles)
print(pred)

img_embeds shape torch.Size([45, 64, 4096])
inputs_embeds shape torch.Size([1, 3034, 4096])
attention_mask shape torch.Size([1, 3034])
This video features a woman and her dog performing various tricks with frisbees, including jumping over each other's back while holding the disk in their mouth. The footage showcases an entertainment event where they exhibit these impressive skills to display how well-trained both are as performers of canine sports such as agility or freestyle flying disc training for dogs that have been taught basic commands like "stay" which enables them not only do this trick but also stay still when needed during performances on stages or at events involving audiences watching from outside observing all actions take place under supervised conditions by trainers/handlers who ensure safety precautions taken into consideration given some may be sensitive about animal welfare concerns associated with activities performed around animals especially those considered pets r